The following jupyter notebook demonstrates how a point file containing elk locations can be combined with other datasets to help understand their habits.

Four questions were asked of us as we analyze this elk data. They include:
1. Are the locations east or west of the continental divide
2. Are the locations within the national park boundary
3. At what elevation are the locations
4. What is the land cover where these locations exist

To help in answering these questions 4 datasets were identified, and all but the extremely large land cover have been saved into the 'data' folder, along with our elk locations data.

Here are the sources of data we'll be using to add the additional columns ('within','elevation','land_cover') to our dataset which will be exported to a new file at the end of this notebook.
* East or west of continental divide - https://hub.arcgis.com/datasets/nps::rocky-mountain-national-park-continental-divide/explore?location=40.243939%2C-104.920569%2C9.00
* Inside park boundary - https://romo-nps.opendata.arcgis.com/datasets/7cb5f22df8c44900a9f6632adb5f96a5/explore?location=40.426948%2C-105.429099%2C10.41
* ~~Elevation (USGS 1/3 Arc Second n41w106 20230314)- https://geodata.colorado.gov/apps/the-national-map-downloader-1/explore~~
* Elevation data accessed from SpatioTemporal Asset Catalog (STAC) - https://planetarycomputer.microsoft.com
* Vegetation - https://www.mrlc.gov/data-services-page

We'll start by visualizing the data allowing us to see what we're working with.
Then we'll show different ways to interate over the rows in our locations data file to inject details from the other datasets.

In [1]:
# Load in the needed libraries. These need to be installed before proceeding

import pandas as pd # To allow easy CSV data loading
import geopandas
import json
# from ipyleaflet import Map, GeoJSON, Marker, MarkerCluster, ImageOverlay, WMSLayer, LayersControl # for creating a map
import folium
from folium.plugins import MarkerCluster

from localtileserver import TileClient, get_folium_tile_layer # to visualize the geotif 
import rasterio # to sample from the geotif

from shapely.geometry import Point, Polygon, shape # for checking points in a polygon

from IPython.display import Image, IFrame, display # to embed an image and web page within the notebook
from ipywidgets import IntProgress

from ipywebrtc import WidgetStream, ImageRecorder # for making an image of the map

import urllib.request # to make a web request for using a web service
import time # to allow a timed break between web service request so as to not overwhelm the server

In [2]:
# Load in the location data (Format CSV)
location_file="data/ElkWinter08_09.csv"
location_df=pd.read_csv(location_file)

In [3]:
# Load in the Continential Divide data (Format GeoJSON)
divide_file="data/Rocky_Mountain_National_Park_-_Continental_Divide-shp.zip"
divide_data = geopandas.read_file(divide_file)

In [4]:
# Load in the park boundary data (Format GeoJSON)
park_file="data/Rocky_Mountain_National_Park_-_Boundary_Polygon.geojson"
with open(park_file, 'r') as f:
    park_data = json.load(f)

In [5]:
# get the elevation data
# go to https://radiantearth.github.io/stac-browser/
'''
Search 'Earth Search' (with ctrl-f), 
Click on the Eath Search catalog,
Click the source button (top right of the page), 
Copy the field 'STAC metadata file is located' and paste below
Correction - the earth-search.aws requires a AWS_SECRET_ACCESS_KEY, so we'll use the mirosoft planetary computer
'''

#api_url='https://earth-search.aws.element84.com/v1/'
api_url="https://planetarycomputer.microsoft.com/api/stac/v1"

from pystac_client import Client
import planetary_computer

client = Client.open(api_url,modifier=planetary_computer.sign_inplace)

In [7]:
# get elevation data continued
'''
Click the item "Copernicus DEM GLO-30"
Click the source button (top right of the page), 
Copy the ID and paste below

'''
collection = 'cop-dem-glo-30'

from shapely.geometry import Point
# rocky mountain national park coordinates - from https://www.google.com/maps
# right click location, copy lat(y) and lng(x), and flip to conform to x,y
point = Point(-105.68988068298488, 40.384733209948294)  

search = client.search(
    collections=[collection],
    intersects=point,
    max_items=10,
)
print(len(search.item_collection()))

1


In [8]:
# get the first (and only) asset from the search
items=search.item_collection()
signed_asset = planetary_computer.sign(items[0].assets["data"])

In [14]:
# Create a tile layer since most browsers don't show Tiff files
elevation_file=signed_asset.href
elevation_tiles = TileClient(elevation_file) # create tiles client
elevation_layer = get_folium_tile_layer(elevation_tiles, name='elevation') # create elevation tile layer


In [10]:
# create a Web Map Service Layer allowing us to stream in this data from the web without having to download it.
land_cover_wms = folium.WmsTileLayer(
    url='https://www.mrlc.gov/geoserver/mrlc_display/NLCD_2021_Land_Cover_L48/wms',
    layers='NLCD_2021_Land_Cover_L48',
    format='image/png',
    name='NLCD_2021_Land_Cover_L48' # add name to appear in layer dislay control
)

# show the legend for this data since the actual values will just be numeric
Image(url= "https://www.mrlc.gov/geoserver/mrlc_display/NLCD_2021_Land_Cover_L48/wms?service=WMS&request=GetLegendGraphic&format=image/png&width=20&height=20&layer=NLCD_2021_Land_Cover_L48")


In [11]:
# With all the data loaded, well let's see what we're working with for our location data
location_df.shape

(10485, 7)

In [16]:
# As the size of our location data too large to simply show on a map (>1000), we'll cluster the locations first
marker_cluster = MarkerCluster(name='Elk locations')

# loop over the locations, convert them to markers for map display
for index, row in location_df.iterrows():
    folium.Marker(location=(row['y'],row['x'])).add_to(marker_cluster)

In [17]:
# create a map to display the locations and other datasets

# center the map over the location of the last marker
map = folium.Map([row['y'],row['x']], zoom_start = 10)

# create the geojson layers
folium.GeoJson(divide_data,
    style_function=lambda feature:
        {'color': 'blue','opacity': 1, 'dashArray': '9', 'fillOpacity': 0.2, 'weight': 4},
               name="Continental Divide"
).add_to(map)

folium.GeoJson(park_data,
    style_function=lambda feature:
        { 'color': 'red','opacity': 1, 'dashArray': '9', 'fillOpacity': 0.2, 'weight': 2},
        name="RMNP park boundary"
).add_to(map)


# add other layers
land_cover_wms.add_to(map)

elevation_layer.add_to(map)

marker_cluster.add_to(map)

# add a layer control
folium.LayerControl().add_to(map)

# show the map inline
map


We'll now use the locations and check each against the appropriate dataset to store the new data as part of the location dataframe.
It should be noted that all the points are seen to be east of the continental divide which answers the first question

In [58]:
# Check if locations are in the park polygon

park_polygon: Polygon = shape(park_data["features"][0]["geometry"])

location_df["within_park"] = ""
for index, row in location_df.iterrows():
    marker=Point(row['x'],row['y'])
    within=marker.within(park_polygon)
    # row['within_park']=within
    location_df.loc[index, 'within_park'] = within

In [59]:
# add elevation information to each row

#create raster layer for sampling
src = rasterio.open(elevation_file)

# create a list of coordinates for sampling
coord_list = [(x, y) for x, y in zip(location_df["x"], location_df["y"])]

# save the samples back to the dataframe, and choose the first value
location_df["elevation"] = [x[0] for x in src.sample(coord_list)]
location_df.head()

,OBJECTID,CollarID,Date_MMDDY,Speed_mph,Heading,y,x,within_park,elevation
0,146467,843COW,3/5/2009,9.2,280,40.367308,-105.664583,True,3315.504150
1,146378,843COW,11/16/2008,1.2,345,40.358716,-105.656541,True,2984.461426
2,146816,856,11/7/2008,0.0,245,40.415933,-105.646525,True,2801.393799
3,68067,881,11/7/2008,0.0,215,40.410008,-105.644883,True,2606.151855
4,146812,856,11/7/2008,0.0,305,40.412341,-105.644525,True,2633.158691


In [ ]:
# add land cover information to each row

def get_url_result(url):
    """
    Attempts to connect to a webpage
    :param url: an endpoint
    :return: the result in json
    """
    attempt = 0
    #print("get_url_result:",url )

    try:
        webURL = urllib.request.urlopen(url)
        data = webURL.read()
        encoding = webURL.info().get_content_charset('utf-8')
        return json.loads(data.decode(encoding))


    except Exception as err:

        if attempt > 1:
            print("More than 3 attempts failed to retrieve file. Aborting.")
            return
        else:
            print("Retrieving file failed, waiting 5 sec")
            print("Message was:", str(err))
            time.sleep(1)

    finally:
        attempt += 1

#allow showing of progress
f = IntProgress(min=0, max=location_df.shape[0]) # instantiate the bar with the number of rows for progress
display(f)


# create raster layer for sampling
for index,row in location_df.iterrows():
    
    lat=location_df.at[index,"y"]
    lng=location_df.at[index,"x"]
    lat2=location_df.at[index,"y"]+0.00001
    lng2=location_df.at[index,"x"]+0.00001
    url = "https://www.mrlc.gov/geoserver/mrlc_display/NLCD_2021_Land_Cover_L48/wms?QUERY_LAYERS=NLCD_2021_Land_Cover_L48&INFO_FORMAT=application/json&x=1&y=1&width=2&height=2&bbox={}%2C{}%2C{}%2C{}&crs=EPSG%3A4326&format=image%2Fpng&request=GetFeatureInfo&service=WMS&styles=&transparent=true&version=1.3.0&layers=NLCD_2021_Land_Cover_L48".format(lat,lng,lat2,lng2)

    f.value = index
    try:
        location_df.loc[index, 'land_cover'] = get_url_result(url)['features'][0]['properties']['PALETTE_INDEX']
    except Exception as err:
        pass
    

In [ ]:
# export the dataframe
location_df.to_csv('processed_date.csv', index=False)